In [ ]:
import pathlib

In [ ]:
from chmp.ds import reload

import asyncio
import datetime
import json
import logging
import pickle

from pathlib import Path

logging.basicConfig(level=logging.DEBUG)

reload('chmp.app.seqdec.sequencing', 'chmp.app.seqdec.cli')
from chmp.app.seqdec.cli import execute
from chmp.app.seqdec.sequencing import find_blocks, build_sequence

In [ ]:
with (Path.cwd() / 'data' / 'sequences' / 'step_meta.json').open('rt') as fobj:
    meta = json.load(fobj)
    
meta = {desc['name']: desc for desc in meta}


with (Path.cwd() / 'data' / 'sequences' / 'manual1.json').open('rt') as fobj:
    sequence, *_ = json.load(fobj)
    
sequence = [meta[pose] for pose in sequence['poses']]
sequence = build_sequence(sequence)
sequence = [dict(p, message=p['message'].format(**p)) for p in sequence]


with (Path.cwd() / '..' / 'KeywordDetection' / 'run' / 'model' / 'model.pickle').open('rb') as fobj:
    keyword_model = pickle.load(fobj)

In [ ]:
loop = asyncio.get_event_loop()
events = execute(sequence, keyword_model, loop=loop)

In [ ]:
fname = Path() / 'data' / 'traces' / datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S.json')
if not fname.exists():
    with fname.open('wt') as fobj:
        json.dump(events, fobj, indent=2, sort_keys=True, ensure_ascii=False)